In [12]:
#data science and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier

#creation of dataset
import _lib.ml_workflow.create_dataset as cds
from _lib.export import to_csv
from _lib.raman_lib.misc import load_data

#quality control
import _lib.ml_workflow.quality_control as qc
from _lib.raman_lib.preprocessing import RangeLimiter
from _lib.raman_lib.visualization import plot_spectra_peaks
from _lib.raman_lib.spectra_scoring import score_names

#preprocessing
from _lib.ml_workflow.preprocess_data import preprocess

#model creation
from sklearn.model_selection import LeaveOneGroupOut, cross_validate, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.cluster import FeatureAgglomeration
from sklearn.decomposition import PCA, NMF
from sklearn.pipeline import Pipeline
from _lib.raman_lib.preprocessing import PeakPicker

#file handling
from pathlib import Path
import os

In [13]:
# define the paths to all experiment data
# each dir contains all the files associated with that experiment
# each file has a prefix that indicates the group and the measurement --> group_measurement_.....
path_etoposide = "/Users/Praktikum/Documents/L540/Etoposide"
path_resveratrol = "/Users/Praktikum/Documents/L540/Resveratrol"
path_both = "/Users/Praktikum/Documents/L540/Both"
path_control = "/Users/Praktikum/Documents/L540/Control"

# Define parameters
In order to function properly, the provided code depends on predefined parameters like output paths, limits and thresholds for the quality control, ...
## Define data paths
Define the location of the data, and where quality-controlled and preprocessed data should be stored. Both of them rely on a unique file-prefix that describes the data being analyzed.

In [14]:
FILE_PREFIX = "HL_540_C_E"
DATASET_OUT = "./GroupK/data/" + FILE_PREFIX + ".csv"
RESULT_DIR = "./GroupK/result/" + FILE_PREFIX
QC_OUT = RESULT_DIR + "/" + FILE_PREFIX + "_qc.csv"
PREP_OUT = RESULT_DIR + "/" + FILE_PREFIX + "_preprocessed.csv"
LDA_DIR = RESULT_DIR + "/" + "lda_dim_reduction"
REG_DIR = RESULT_DIR + "/" + "regularized_models/"
TREE_DIR = RESULT_DIR + "/" + "tree_based_models/"

## Define quality scoring parameters
The quality control only uses peaks in a given interval, recognizes peaks via a filter (Sav-Gol) and scores them based on some metrics. Finally, the best N spectra are selected.
### Spectral Range Limits

In [15]:
QC_LIM_LOW = 450
QC_LIM_HIGH = 1650

### Peak Detection

In [16]:
QC_WINDOW = 35
QC_THRESHOLD = 0.001
QC_MIN_HEIGHT = 50

### Scoring

In [17]:
QC_SCORE = 1
QC_PEAKS = 1

### Number of spectra to keep

In [18]:
QC_NUM = 300

## Define Preprocessing Parameter
### Spectral Range Limits

In [19]:
PREP_LIM_LOW = QC_LIM_LOW
PREP_LIM_HIGH = QC_LIM_HIGH

### Window-width for smoothing

In [20]:
PREP_WINDOW = 15

## Settings for Cross Validation

In [21]:
SCORING = ['roc_auc', 'accuracy', 'f1']
N_TRIALS = 20
N_FOLDS = 5
N_CORES = -1

# Create the dataset
Create the dataset using the implementation provided by D. Zimmermann.
For the creation of the dataset, the two source dirs, as well as the desired labels are needed.
Furthermore, an output directory is needed, to store the created dataset 

In [22]:
datadir = Path(DATASET_OUT).parent
if not os.path.exists(datadir):
    os.makedirs(datadir)

dataset = cds.create_dataset([path_control, path_etoposide], ['Control', 'Etoposide'], grouped=True)
dataset.to_csv(DATASET_OUT, index=False)

root - INFO - Loading data
root - INFO - Loading data
root - INFO - Loading data
root - INFO - Loading files from /Users/Praktikum/Documents/L540/Control
root - INFO - Loading files from /Users/Praktikum/Documents/L540/Control
root - INFO - Loading files from /Users/Praktikum/Documents/L540/Control
root - INFO - Loading files from /Users/Praktikum/Documents/L540/Etoposide
root - INFO - Loading files from /Users/Praktikum/Documents/L540/Etoposide
root - INFO - Loading files from /Users/Praktikum/Documents/L540/Etoposide
root - INFO - Finished loading data.
root - INFO - Finished loading data.
root - INFO - Finished loading data.


# Do quality control
Asses the spectra based on their quality, and remove low quality spectra

In [23]:
path_in = Path(DATASET_OUT)
path_out = Path(RESULT_DIR)

if not os.path.exists(path_out):
    os.makedirs(path_out)

path_out_data = path_out / (path_in.stem + "_qc.csv")
path_out_scores = path_out / (path_in.stem + "_qc_scores.csv")

data = pd.read_csv(path_in)

data_out, _, score_dict = qc.score_sort_spectra(data,
                                                n=QC_NUM,
                                                limits=[QC_LIM_LOW, QC_LIM_HIGH],
                                                bl_method="asls",
                                                sg_window=QC_WINDOW,
                                                threshold=QC_THRESHOLD,
                                                min_height=QC_MIN_HEIGHT,
                                                score_measure=QC_SCORE,
                                                n_peaks_influence=QC_PEAKS,
                                                detailed=True)

visualize = False
if visualize:
    data_vis = data.drop(columns=["label", "file", "group"]).values.astype(float)
    wns_vis = data.drop(columns=["label", "file", "group"]).columns.astype(float)

    rl = RangeLimiter(lim=[QC_LIM_LOW, QC_LIM_HIGH],
                      reference=wns_vis)

    data_rl = rl.fit_transform(data_vis)
    wns_rl = wns_vis[rl.lim_[0]:rl.lim_[1]]

    plot_spectra_peaks(wns_rl,
                       data_rl,
                       score_dict["peak_pos"],
                       labels=score_dict["total_scores"])

data_out.to_csv(path_out_data, index=False)

pd.DataFrame({score_names[QC_SCORE]: score_dict["intensity_scores"],
              "N Peaks": score_dict["peak_scores"]}).to_csv(
    path_out_scores, index=False
)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Preprocess the data

In [24]:
path_in = Path(QC_OUT)
path_out = Path(RESULT_DIR)

filename = path_in.stem.removesuffix("_qc")

if not os.path.exists(path_out):
    os.makedirs(path_out)

path_out = path_out / (filename + "_preprocessed.csv")

data = load_data(QC_OUT)

# save groups and remove column, otherwise preprocess won't work
groups = np.asarray(data.group)
data = data.drop(columns=["group"])

data_prep = preprocess(data, limits=[PREP_LIM_LOW, PREP_LIM_HIGH], sg_window=PREP_WINDOW)

#add groups again
data_prep.insert(2, "group", groups)

data_prep.to_csv(path_out, index=False)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Implement GroupKFold CV
## LDA Dimensionality Reduction 

In [25]:
path_in = PREP_OUT
#path_out = Path(args.out)

#filename = path_in.stem

data = load_data(path_in)

X = data.drop(columns=["label", "file", "group"])
wns = np.asarray(X.columns.astype(float))
X = np.asarray(X)
y = np.array(data.label)
y, y_key = pd.factorize(y)

#split the dataset according to the groups
groups = np.array(data.group)
logo = LeaveOneGroupOut()



### Baseline with LDA alone

In [26]:
clf = LinearDiscriminantAnalysis()
result = cross_validate(clf, X, y, cv=logo, scoring=SCORING, return_train_score=True, groups=groups)
to_csv(result, path=LDA_DIR + "/lda", scoring=SCORING, param_opt=False)
result

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


{'fit_time': array([0.38143063, 0.20204282, 0.19964194]),
 'score_time': array([0.00563097, 0.00352001, 0.00328994]),
 'test_roc_auc': array([0.60699649, 0.71925287, 0.74025024]),
 'train_roc_auc': array([1.        , 0.99997704, 1.        ]),
 'test_accuracy': array([0.57225434, 0.70224719, 0.74297189]),
 'train_accuracy': array([0.99765808, 0.99526066, 1.        ]),
 'test_f1': array([0.53164557, 0.51376147, 0.82887701]),
 'train_f1': array([0.99790356, 0.99586777, 1.        ])}

### Feature Selection with PCA followed by LDA

In [27]:
param_grid = {"pca__n_components": range(
    1, 51, 10
)}
clf = Pipeline([("pca", PCA()),
                ("lda", LinearDiscriminantAnalysis())])
grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy',
                       return_train_score=True, verbose=3).fit(X, y, groups=groups)

to_csv(grid_rf.cv_results_, scoring=SCORING, path=LDA_DIR + "/pca_lda", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)


Fitting 3 folds for each of 5 candidates, totalling 15 fits
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel

### Non-Negative Matrix Factorization

In [28]:
"""clf = Pipeline([("nmf", NMF(init="nndsvda", tol=1e-2, max_iter=5000)),
                ("lda", LinearDiscriminantAnalysis())])
cross_validate(clf, X, y, cv=logo, groups=groups, scoring=SCORING)"""

'clf = Pipeline([("nmf", NMF(init="nndsvda", tol=1e-2, max_iter=5000)),\n                ("lda", LinearDiscriminantAnalysis())])\ncross_validate(clf, X, y, cv=logo, groups=groups, scoring=SCORING)'

### Feature Agglomeration

In [29]:
param_grid = {"agglo__n_clusters": range(
    5, 41, 5
)}
clf = Pipeline([("agglo", FeatureAgglomeration(connectivity=np.diag(np.ones(len(wns))) +
                                                            np.diag(np.ones(len(wns) - 1), 1) +
                                                            np.diag(np.ones(len(wns) - 1), -1))),
                ("lda", LinearDiscriminantAnalysis())])
grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy',
                       return_train_score=True, verbose=3).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=LDA_DIR + "/fa_lda", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV 1/3] END agglo__n_clusters=5; accuracy: (train=0.808, test=0.751) f1: (train=0.824, test=0.606) roc_auc: (train=0.908, test=0.842) total time=   0.2s
[CV 2/3] END agglo__n_clusters=5; accuracy: (train=0.732, test=0.697) f1: (train=0.770, test=0.518) roc_auc: (train=0.811, test=0.722) total time=   0.2s
[CV 3/3] END agglo__n_clusters=5; accuracy: (train=0.852, test=0.791) f1: (train=0.782, test=0.869) roc_auc: (train=0.906, test=0.834) total time=   0.2s
[CV 1/3] END agglo__n_clusters=10; accuracy: (train=0.857, test=0.832) f1: (train=0.874, test=0.797) roc_auc: (train=0.938, test=0.947) total time=   0.2s
[CV 2/3] END agglo__n_clusters=10; accuracy: (train=0.836, test=0.725) f1: (train=0.852, test=0.449) roc_auc: (train=0.914, test=0.798) total time=   0.2s
[CV 3/3] END agglo__n_clusters=10; accuracy: (train=0.869, test=0.570) f1: (train=0.802, test=0.665) roc_auc: (train=0.929, test=0.601) total time=   0.3s
[CV 1/3] END 

### PeakPicker

In [30]:
param_grid = {"peaks__min_dist": range(
    10, 151, 10
)}
clf = Pipeline([("peaks", PeakPicker()),
                ("lda", LinearDiscriminantAnalysis())])
grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=LDA_DIR + "/peak_lda", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)


Fitting 3 folds for each of 15 candidates, totalling 45 fits
[CV 1/3] END peaks__min_dist=10; accuracy: (train=0.871, test=0.815) f1: (train=0.886, test=0.765) roc_auc: (train=0.946, test=0.927) total time=   0.0s
[CV 2/3] END peaks__min_dist=10; accuracy: (train=0.905, test=0.820) f1: (train=0.919, test=0.619) roc_auc: (train=0.978, test=0.914) total time=   0.0s
[CV 3/3] END peaks__min_dist=10; accuracy: (train=0.877, test=0.707) f1: (train=0.820, test=0.815) roc_auc: (train=0.947, test=0.609) total time=   0.0s
[CV 1/3] END peaks__min_dist=20; accuracy: (train=0.881, test=0.827) f1: (train=0.894, test=0.783) roc_auc: (train=0.945, test=0.928) total time=   0.0s
[CV 2/3] END peaks__min_dist=20; accuracy: (train=0.905, test=0.815) f1: (train=0.920, test=0.612) roc_auc: (train=0.976, test=0.916) total time=   0.0s
[CV 3/3] END peaks__min_dist=20; accuracy: (train=0.866, test=0.707) f1: (train=0.807, test=0.815) roc_auc: (train=0.945, test=0.602) total time=   0.0s
[CV 1/3] END peaks__m

## Regularized Models
### Logistic Regression L1 Penalty

In [31]:
param_grid = {
    "logreg__C": np.logspace(-2, 1, 16)
}
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(solver="liblinear",
                                  penalty="l1", max_iter=1000, random_state=41))
])

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=REG_DIR + "/logreg_l1", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV 1/3] END logreg__C=0.01; accuracy: (train=0.857, test=0.873) f1: (train=0.872, test=0.788) roc_auc: (train=0.939, test=0.940) total time=   0.1s
[CV 2/3] END logreg__C=0.01; accuracy: (train=0.860, test=0.798) f1: (train=0.881, test=0.571) roc_auc: (train=0.935, test=0.894) total time=   0.1s
[CV 3/3] END logreg__C=0.01; accuracy: (train=0.746, test=0.739) f1: (train=0.708, test=0.848) roc_auc: (train=0.907, test=0.794) total time=   0.1s
[CV 1/3] END logreg__C=0.015848931924611134; accuracy: (train=0.874, test=0.815) f1: (train=0.883, test=0.660) roc_auc: (train=0.942, test=0.940) total time=   0.0s
[CV 2/3] END logreg__C=0.015848931924611134; accuracy: (train=0.874, test=0.792) f1: (train=0.890, test=0.554) roc_auc: (train=0.949, test=0.895) total time=   0.1s
[CV 3/3] END logreg__C=0.015848931924611134; accuracy: (train=0.746, test=0.743) f1: (train=0.708, test=0.850) roc_auc: (train=0.907, test=0.791) total time=   0.

### Logistic Regression L2 Penalty

In [32]:
param_grid = {
    "logreg__C": np.logspace(-5, 1, 13)
}
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(solver="liblinear",
                                  penalty="l2", max_iter=1000, random_state=51))
])

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=REG_DIR + "/logreg_l2", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 3 folds for each of 13 candidates, totalling 39 fits
[CV 1/3] END logreg__C=1e-05; accuracy: (train=0.815, test=0.757) f1: (train=0.818, test=0.488) roc_auc: (train=0.915, test=0.891) total time=   0.0s
[CV 2/3] END logreg__C=1e-05; accuracy: (train=0.829, test=0.691) f1: (train=0.835, test=0.203) roc_auc: (train=0.924, test=0.761) total time=   0.0s
[CV 3/3] END logreg__C=1e-05; accuracy: (train=0.835, test=0.731) f1: (train=0.796, test=0.816) roc_auc: (train=0.932, test=0.740) total time=   0.0s
[CV 1/3] END logreg__C=3.1622776601683795e-05; accuracy: (train=0.824, test=0.780) f1: (train=0.827, test=0.558) roc_auc: (train=0.923, test=0.913) total time=   0.0s
[CV 2/3] END logreg__C=3.1622776601683795e-05; accuracy: (train=0.839, test=0.697) f1: (train=0.845, test=0.229) roc_auc: (train=0.937, test=0.788) total time=   0.0s
[CV 3/3] END logreg__C=3.1622776601683795e-05; accuracy: (train=0.835, test=0.735) f1: (train=0.796, test=0.823) roc_auc: (train=0.942, test=0.763) total t

### Linear SVM L1 Penalty

In [33]:
param_grid = {
    "svm__C": np.logspace(-3, 0, 16)
}
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", LinearSVC(penalty="l1", dual=False, max_iter=10000))
])

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=REG_DIR + "/svm_l1", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV 1/3] END svm__C=0.001; accuracy: (train=0.440, test=0.647) f1: (train=0.000, test=0.000) roc_auc: (train=0.500, test=0.500) total time=   0.0s
[CV 2/3] END svm__C=0.001; accuracy: (train=0.427, test=0.674) f1: (train=0.000, test=0.000) roc_auc: (train=0.500, test=0.500) total time=   0.0s
[CV 3/3] END svm__C=0.001; accuracy: (train=0.661, test=0.273) f1: (train=0.000, test=0.000) roc_auc: (train=0.500, test=0.500) total time=   0.0s
[CV 1/3] END svm__C=0.001584893192461114; accuracy: (train=0.440, test=0.647) f1: (train=0.000, test=0.000) roc_auc: (train=0.500, test=0.500) total time=   0.0s
[CV 2/3] END svm__C=0.001584893192461114; accuracy: (train=0.427, test=0.674) f1: (train=0.000, test=0.000) roc_auc: (train=0.500, test=0.500) total time=   0.0s
[CV 3/3] END svm__C=0.001584893192461114; accuracy: (train=0.661, test=0.273) f1: (train=0.000, test=0.000) roc_auc: (train=0.500, test=0.500) total time=   0.0s
[CV 1/3] END

### Linear SVM L2 Penalty

In [34]:
param_grid = {
    "svm__C": np.logspace(-5, -1, 13)
}
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", LinearSVC(penalty="l2", max_iter=5000))
])

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=REG_DIR + "/svm_l2", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 3 folds for each of 13 candidates, totalling 39 fits
[CV 1/3] END svm__C=1e-05; accuracy: (train=0.850, test=0.821) f1: (train=0.855, test=0.680) roc_auc: (train=0.937, test=0.935) total time=   0.0s
[CV 2/3] END svm__C=1e-05; accuracy: (train=0.870, test=0.713) f1: (train=0.878, test=0.282) roc_auc: (train=0.956, test=0.824) total time=   0.1s
[CV 3/3] END svm__C=1e-05; accuracy: (train=0.840, test=0.767) f1: (train=0.800, test=0.851) roc_auc: (train=0.957, test=0.792) total time=   0.0s
[CV 1/3] END svm__C=2.1544346900318823e-05; accuracy: (train=0.867, test=0.861) f1: (train=0.874, test=0.782) roc_auc: (train=0.949, test=0.948) total time=   0.0s
[CV 2/3] END svm__C=2.1544346900318823e-05; accuracy: (train=0.882, test=0.702) f1: (train=0.890, test=0.232) roc_auc: (train=0.970, test=0.855) total time=   0.0s
[CV 3/3] END svm__C=2.1544346900318823e-05; accuracy: (train=0.855, test=0.771) f1: (train=0.819, test=0.856) roc_auc: (train=0.970, test=0.814) total time=   0.0s
[CV 1/

## Tree-based models
### Basic Decision Tree


In [35]:
param_grid = {
    "ccp_alpha": np.logspace(-3, -1, 9)
}
clf = DecisionTreeClassifier(random_state=653)

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=TREE_DIR + "/decision_tree", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV 1/3] END ccp_alpha=0.001; accuracy: (train=1.000, test=0.665) f1: (train=1.000, test=0.540) roc_auc: (train=1.000, test=0.640) total time=   0.7s
[CV 2/3] END ccp_alpha=0.001; accuracy: (train=1.000, test=0.730) f1: (train=1.000, test=0.500) roc_auc: (train=1.000, test=0.649) total time=   0.4s
[CV 3/3] END ccp_alpha=0.001; accuracy: (train=1.000, test=0.482) f1: (train=1.000, test=0.596) roc_auc: (train=1.000, test=0.446) total time=   0.5s
[CV 1/3] END ccp_alpha=0.0017782794100389228; accuracy: (train=1.000, test=0.665) f1: (train=1.000, test=0.540) roc_auc: (train=1.000, test=0.640) total time=   0.7s
[CV 2/3] END ccp_alpha=0.0017782794100389228; accuracy: (train=1.000, test=0.730) f1: (train=1.000, test=0.500) roc_auc: (train=1.000, test=0.649) total time=   0.4s
[CV 3/3] END ccp_alpha=0.0017782794100389228; accuracy: (train=1.000, test=0.482) f1: (train=1.000, test=0.596) roc_auc: (train=1.000, test=0.446) total time=

### Random Forest

In [36]:
param_grid = {
    "colsample_bytree": np.linspace(0.01, 0.2, 20)
}
clf = LGBMClassifier(boosting_type="rf",
                     subsample=0.8,
                     subsample_freq=1,
                     max_bin=10,
                     max_depth=8,
                     random_state=2434)

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=TREE_DIR + "/random_forest", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV 1/3] END colsample_bytree=0.01; accuracy: (train=0.939, test=0.832) f1: (train=0.945, test=0.713) roc_auc: (train=0.989, test=0.922) total time=   0.2s
[CV 2/3] END colsample_bytree=0.01; accuracy: (train=0.948, test=0.781) f1: (train=0.954, test=0.598) roc_auc: (train=0.991, test=0.830) total time=   0.1s
[CV 3/3] END colsample_bytree=0.01; accuracy: (train=0.920, test=0.478) f1: (train=0.867, test=0.504) roc_auc: (train=0.993, test=0.633) total time=   0.3s
[CV 1/3] END colsample_bytree=0.02; accuracy: (train=0.930, test=0.844) f1: (train=0.935, test=0.722) roc_auc: (train=0.985, test=0.955) total time=   0.2s
[CV 2/3] END colsample_bytree=0.02; accuracy: (train=0.943, test=0.787) f1: (train=0.950, test=0.568) roc_auc: (train=0.990, test=0.829) total time=   0.2s
[CV 3/3] END colsample_bytree=0.02; accuracy: (train=0.929, test=0.590) f1: (train=0.884, test=0.673) roc_auc: (train=0.986, test=0.659) total time=   0.2s
[CV

### Gradient-boosted Decision Tree

In [37]:
param_grid = {
    "learning_rate": np.linspace(0.01, 0.2, 20)
}
clf = LGBMClassifier(colsample_bytree=0.2,
                     max_bin=10,
                     max_depth=5,
                     random_state=6233)

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=TREE_DIR + "/gbdt", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV 1/3] END learning_rate=0.01; accuracy: (train=0.965, test=0.867) f1: (train=0.969, test=0.781) roc_auc: (train=0.996, test=0.922) total time=   0.5s
[CV 2/3] END learning_rate=0.01; accuracy: (train=0.955, test=0.803) f1: (train=0.961, test=0.598) roc_auc: (train=0.995, test=0.773) total time=   0.3s
[CV 3/3] END learning_rate=0.01; accuracy: (train=0.934, test=0.454) f1: (train=0.893, test=0.524) roc_auc: (train=0.995, test=0.504) total time=   0.3s
[CV 1/3] END learning_rate=0.02; accuracy: (train=0.995, test=0.832) f1: (train=0.996, test=0.701) roc_auc: (train=1.000, test=0.949) total time=   0.5s
[CV 2/3] END learning_rate=0.02; accuracy: (train=0.983, test=0.803) f1: (train=0.985, test=0.568) roc_auc: (train=0.999, test=0.806) total time=   0.4s
[CV 3/3] END learning_rate=0.02; accuracy: (train=0.989, test=0.518) f1: (train=0.983, test=0.623) roc_auc: (train=1.000, test=0.510) total time=   0.4s
[CV 1/3] END learning